# Purpose 
 
The present verification report aims at checking the post-processing of the convective, diffusive and source terms of the standard and realisable $k$-$\varepsilon$ transport equations using VDF and VEF discretizations. No numerical resolution is performed.

Validation made by: Pierre-Emmanuel Angeli and Alan Burlot

# Problem Description 

## Geometry 

The computational domain is the unit cube.

## Initial Conditions and Boundary Conditions 

- **Boundary conditions**: Triple periodicity.


- **Initial conditions**: Periodic velocity, turbulent kinetic energy and turbulent dissipation fields defined by the following analytical expressions:

\begin{align*}
&\text{velocity vector: } && \textbf{u}(x,y,z) = \begin{pmatrix} -2 \cos(2\pi x) \sin(2\pi y) \sin(2\pi z) \\\\ \sin(2\pi x) \cos(2\pi y) \sin(2\pi z) \\\\ \sin(2\pi x) \sin(2\pi y) \cos(2\pi z) \end{pmatrix} \\ 
&\text{turbulent kinetic energy: } && k(x,y,z) = \cos(2 \pi y)+2 \\ 
&\text{turbulent dissipation rate: } && \varepsilon(x,y,z) = \sin(2 \pi y)+2\end{align*}

N.B.: The imposed velocity field has a zero divergence.

## Fluid Properties 

- $\mu$ = 0.001 Pa.s
- $\rho$ = 1000 kg/s

## Flow Physics 

This is a verification of post-processing: analytical velocity, $k$ and $\varepsilon$ fields are imposed and frozen (no resolution of the flow).

# Case Setup 

## Grid 

- The VDF mesh has 101 points in each direction $\Rightarrow (101-1)^3 = 10^6$ cells.
- The VEF mesh has 26 points in each direction and the cells are divided into 48 tetrahedra with the ```tetraedriser_homogene_fin``` option $\Rightarrow 48 \times (26-1)^3 = 7.5 \cdot 10^5$ cells.


## Model Options 


The transport equations of $k$ and $\varepsilon$ write under the following form:


\begin{align*}
&\dfrac{\partial k}{\partial t} + \underbrace{U_i \dfrac{\partial k}{\partial x_i}}_{\text{conv}_k} = \underbrace{\dfrac{\partial}{\partial x_i} \left[ \left( \dfrac{\nu_t}{\sigma_k} \right) \dfrac{\partial k}{\partial x_i} \right]}_{\text{diff}_k} + \underbrace{P - \varepsilon}_{\text{ source}_k}\\
&\dfrac{\partial \varepsilon}{\partial t} + \underbrace{U_i \dfrac{\partial \varepsilon}{\partial x_i}}_{\text{conv}_\varepsilon} = \underbrace{\dfrac{\partial}{\partial x_i} \left[ \left( \dfrac{\nu_t}{\sigma_\varepsilon} \right) \dfrac{\partial \varepsilon}{\partial x_i} \right]}_{\text{diff}_\varepsilon} + \text{ source}_\varepsilon
\end{align*}

The $\varepsilon$ source term in the standard and realizable $k$--$\varepsilon$ models is defined as:

\begin{align*}
&\text{standard: } && \text{source}_\varepsilon = \dfrac{\varepsilon}{k} \left( C_{\varepsilon_1} P - C_{\varepsilon_2} \varepsilon \right) \\
&\text{realizable: } && \text{source}_\varepsilon = C_1 S \varepsilon -C_2 \dfrac{\varepsilon^2}{k + \sqrt{\nu \varepsilon}}
\end{align*}

For the complete description of the models, refer to equations and (21) and (54) of [1].


The analytical expressions are computing using symbolic computation with sympy.


In order to get rid of interpolation of their values, the terms are plotted by means of probes at their real location, namely:
- at gravity center of cells in VDF;
- at face center of cells in VEF.

## Other Options (calculation) 

The ```equation_non_resolue``` option is used to disable the resolution.


The number of time steps ```nb_pas_dt_max``` is set to zero.


# Analytic $k$-$\varepsilon$ computation

## Tools and definitions

We define a gradient function and a divergence function with sympy. The `create_function` function allows to convert the sympy expression in a callable function by lambdifying it.

In [ ]:
import sympy
from sympy import cos, sin, pi
import numpy
import pandas


x, y, z = sympy.symbols('x y z', real=True)


def grad(f):
    """Compute the gradient of a scalar function f with respect to x, y, and z."""
    return [sympy.diff(f, x), sympy.diff(f, y), sympy.diff(f, z)]


def div(f):
    """Compute the divergence of a vector field f."""
    return sympy.diff(f[0], x) + sympy.diff(f[1], y) + sympy.diff(f[2], z)


def create_function(func, dsubsti):
    """Lambdify the sympy expression."""
    lam_func = sympy.lambdify([x, y, z],
                              func.subs(dsubsti),
                              modules=[{'sqrt': numpy.emath.sqrt, "Max": numpy.max}, "numpy"])
    return lam_func


Special functions are defined for tensors. The rotation tensor computation was validated to obtain the correct sign.

In [ ]:
def compute_strain_tensor(velocity):
    """Compute 1/2*(du_i/dx_j + du_j/dx_i)."""
    gradv = [grad(velocity[0]),
             grad(velocity[1]),
             grad(velocity[2])]
    strain_ij = pandas.DataFrame((pandas.DataFrame(gradv)
                                  + pandas.DataFrame(gradv).T)/2).values.tolist()
    assert numpy.array(strain_ij).shape == (3, 3)
    assert strain_ij[0][1] == strain_ij[1][0]
    assert strain_ij[0][2] == strain_ij[2][0]
    assert strain_ij[1][2] == strain_ij[2][1]

    return strain_ij

def compute_rotation_tensor(velocity):
    """Compute 1/2*(du_i/dx_j - du_j/dx_i)."""
    gradv = [grad(velocity[0]),
             grad(velocity[1]),
             grad(velocity[2])]
    rota_ij = pandas.DataFrame((pandas.DataFrame(gradv).T
                                - pandas.DataFrame(gradv))/2).values.tolist()
    assert numpy.array(rota_ij).shape == (3, 3)
    assert rota_ij[0][1] == - rota_ij[1][0]
    assert rota_ij[0][2] == - rota_ij[2][0]
    assert rota_ij[1][2] == - rota_ij[2][1]
    for diag in [0, 1, 2]:
        assert rota_ij[diag][diag] == 0

    return rota_ij

def compute_tensor_contraction(tensor_ij):
    """Compute einstein summation of TijTij"""
    return numpy.einsum('ij,ij->', numpy.array(tensor_ij), numpy.array(tensor_ij))


## Commons

In [ ]:
def compute_convection(velocity, tkeOReps):
    """Compute U_i x dk/dx_i."""
    return sum([a*b for a, b in zip(velocity, grad(tkeOReps))])


def compute_diffusion(tkeOReps, nut):
    """Compute d/dx_i(nu_t/epsX x dX/dx_i)"""
    return div([nut*sympy.diff(tkeOReps, a) for a in [x, y, z]])

## Definition of the models terms

In [ ]:
def compute_production_tke(nut, strain):
    """Compute the production term of the turbulent kinetic energy equation."""
    return 2*nut*strain

def compute_sources_tke(prod, eps):
    """Compute the total source termes for the turbulent kinetic energy of the standard k-epsilon
    equation, i.e. Prod - eps."""
    return prod - eps

def compute_sources_eps_std(kepsstd):
    """Compute the total source termes for the turbulent kinetic energy of the standard k-epsilon
    equation, i.e. eps/tke*(C_eps1*Prod - C_eps2*eps)."""
    return kepsstd["eps"]/kepsstd["tke"] * (kepsstd["ceps1"]*kepsstd["prod_tke"]
                                            - kepsstd["ceps2"]*kepsstd["eps"])

def compute_sources_eps_rea(kepsrea):
    """Compute the total source termes for the turbulent kinetic energy of the standard k-epsilon
    equation, i.e. c1*strain*eps - c2*eps*eps/(tke + sympy.sqrt(nu*eps))."""
    tke, eps = kepsrea["tke"], kepsrea["eps"]
    S = kepsrea["S"]
    c1, c2, nu = kepsrea["c1"], kepsrea["c2"], kepsrea["nu"]
    return c1*S*eps - c2*eps*eps/(tke + sympy.sqrt(nu*eps))


## Computation of the standard $k$-$\varepsilon$ model

In [ ]:
def compute_kepsilon_std():
    """Sympy expressions for the standard k-epsilon."""
    sigmak, sigmae = sympy.symbols("sigmak sigmae")
    ceps1, ceps2 = sympy.symbols('ceps1 ceps2')
    cmu = sympy.symbols("cmu")
    # nu = sympy.symbols("nu")

    # for substitution at the end
    dsubsti = {
        "sigmak": 1,
        "sigmae": 1.3,
        "ceps1": 1.44,
        "ceps2": 1.92,
        "cmu": 0.09
    }

    # init variables
    velocity = [-2*cos(2*pi*x)*sin(2*pi*y)*sin(2*pi*z),
                sin(2*pi*x)*cos(2*pi*y)*sin(2*pi*z),
                sin(2*pi*x)*sin(2*pi*y)*cos(2*pi*z)]
    tke = 2 + cos(2*pi*y)
    eps = 2 + sin(2*pi*y)

    kepsstd = {
        "tke": tke,
        "eps": eps,
        "ceps1": ceps1,
        "ceps2": ceps2
    }

    # Compute the strain tensor and its contracted product
    strain_ij = compute_strain_tensor(velocity)
    strain = compute_tensor_contraction(strain_ij)

    # Turbulent viscosity
    nut = cmu*tke*tke/eps
    kepsstd["nut"] = nut
    
    # TKE
    conv_tke = compute_convection(velocity, tke)
    kepsstd["conv_tke"] = conv_tke
    diff_tke = compute_diffusion(tke, nut/sigmak)
    kepsstd["diff_tke"] = diff_tke
    prod_tke = compute_production_tke(nut, strain)
    kepsstd["prod_tke"] = prod_tke
    sources_tke = compute_sources_tke(prod_tke, eps)
    kepsstd["sources_tke"] = sources_tke

    # EPS
    conv_eps = compute_convection(velocity, eps)
    kepsstd["conv_eps"] = conv_eps
    diff_eps = compute_diffusion(eps, nut/sigmae)
    kepsstd["diff_eps"] = diff_eps
    sources_eps = compute_sources_eps_std(kepsstd)
    kepsstd["sources_eps"] = sources_eps

    return kepsstd

kepsstd = compute_kepsilon_std()

## Computation of the realizable $k$-$\varepsilon$ model

In [ ]:
def compute_kepsilon_rea():
    """Sympy expressions for the realisable k-epsilon model."""

    # Constants of the models
    sigmak = 1.0  # sympy.symbols("sigmak", positive=True, real=True, zero=False)
    sigmae = 1.3  # sympy.symbols("sigmae", positive=True, real=True, zero=False)
    c2 = 1.9  # sympy.symbols("c2", positive=True, real=True, zero=False)
    A0 = 4.04  # sympy.symbols("A0", positive=True, real=True, zero=False)
    nu = 1e-6  # sympy.symbols("nu", positive=True, real=True, zero=False)

    # init fields and constants
    velocity = [-2*cos(2*pi*x)*sin(2*pi*y)*sin(2*pi*z),
                sin(2*pi*x)*cos(2*pi*y)*sin(2*pi*z),
                sin(2*pi*x)*sin(2*pi*y)*cos(2*pi*z)]
    tke = 2 + cos(2*pi*y)
    eps = 2 + sin(2*pi*y)

    dsubsti = {
        "sigmak": 1.0,
        "sigmae": 1.3, 
        "A0": 4.04,
        "c2": 1.9,
        "nu": 1e-6 
    }

    # Create the output dictionnary
    kepsrea = {
        "tke": tke,
        "eps": eps,
        "c2": c2,
        "nu": nu,
        "dsubsti": dsubsti
    }

    # Compute the strain tensor and its contracted product
    strain_ij = compute_strain_tensor(velocity)
    strain = compute_tensor_contraction(strain_ij)
    kepsrea["strain"] = strain

    rota_ij = compute_rotation_tensor(velocity)
    rota = compute_tensor_contraction(rota_ij)
    kepsrea["rota"] = rota

    S = sympy.sqrt(2*strain)
    W = 2**(3/2)*numpy.einsum('ij,jk,ki->', strain_ij, strain_ij, strain_ij)/S**3
    AS = numpy.sqrt(6)*sympy.cos(1/3*sympy.acos(numpy.sqrt(6)*W))
    Ustar = sympy.sqrt(strain + rota)
    cmu = 1/(A0 + AS*Ustar*tke/eps)
    nut = cmu*tke*tke/eps
    C1 = sympy.Max(0.43, S*tke/(5*eps + S*tke))

    kepsrea["S"] = S
    kepsrea["c1"] = C1
    kepsrea["W"] = W
    kepsrea["AS"] = AS
    kepsrea["Ustar"] = Ustar
    kepsrea["cmu"] = cmu

    # Turbulent viscosity
    kepsrea["nut"] = nut

    # TKE
    conv_tke = compute_convection(velocity, tke)
    kepsrea["conv_tke"] = conv_tke
    diff_tke = compute_diffusion(tke, nut)/sigmak
    kepsrea["diff_tke"] = diff_tke
    prod_tke = compute_production_tke(nut, strain)
    kepsrea["prod_tke"] = prod_tke
    sources_tke = compute_sources_tke(prod_tke, eps)
    kepsrea["sources_tke"] = sources_tke

    # EPS
    conv_eps = compute_convection(velocity, eps)
    kepsrea["conv_eps"] = conv_eps
    diff_eps = compute_diffusion(eps, nut)/sigmae
    kepsrea["diff_eps"] = diff_eps
    sources_eps = compute_sources_eps_rea(kepsrea)
    kepsrea["sources_eps"] = sources_eps

    return kepsrea

kepsrea = compute_kepsilon_rea()

## Lambdify functions

We create callable function to plot them.

### $k$-$\varepsilon$ standard

In [ ]:
dsubsti = {
    "sigmak": 1,
    "sigmae": 1.3,
    "ceps1": 1.44,
    "ceps2": 1.92,
    "cmu": 0.09
}
nut_std = create_function(kepsstd["nut"], dsubsti)

# TKE
conv_tke_std = create_function(kepsstd["conv_tke"], dsubsti)
diff_tke_std = create_function(kepsstd["diff_tke"], dsubsti)
prod_tke_std = create_function(kepsstd["prod_tke"], dsubsti)
sources_tke_std = create_function(kepsstd["sources_tke"], dsubsti)

# EPS
conv_eps_std = create_function(kepsstd["conv_eps"], dsubsti)
diff_eps_std = create_function(kepsstd["diff_eps"], dsubsti)
sources_eps_std = create_function(kepsstd["sources_eps"], dsubsti)

### $k$-$\varepsilon$ realisable

In [ ]:
dsubsti = {
    "sigmak": 1.0,
    "sigmae": 1.3,
    "A0": 4.04,
    "c2": 1.9,
    "nu": 1e-6
}
nut_rea = create_function(kepsrea["nut"], dsubsti)

# TKE
conv_tke_rea = create_function(kepsrea["conv_tke"], dsubsti)
diff_tke_rea = create_function(kepsrea["diff_tke"], dsubsti)
prod_tke_rea = create_function(kepsrea["prod_tke"], dsubsti)
sources_tke_rea = create_function(kepsrea["sources_tke"], dsubsti)

# EPS
conv_eps_rea = create_function(kepsrea["conv_eps"], dsubsti)
diff_eps_rea = create_function(kepsrea["diff_eps"], dsubsti)
sources_eps_rea = create_function(kepsrea["sources_eps"], dsubsti)

# TrioCFD computation

In [ ]:
from trustutils import run
 
run.reset() 
run.addCase("VDF_KEPS_STANDARD","Cas.data")
run.addCase("VDF_KEPS_REALISABLE","Cas.data")
run.addCase("VEF_KEPS_STANDARD","Cas.data")
run.addCase("VEF_KEPS_REALISABLE","Cas.data")
run.printCases()
run.runCases()

## Performance Chart

In [ ]:
run.tablePerf()

# Plot results

## VDF standard $k$-$\varepsilon$ model

### Turbulent viscosity

In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Turbulent viscosity of the standard $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_STANDARD/nut.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile")

xval = numpy.linspace(0, 1, 500)
yval = nut_std(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy")

fig.label("y","Turbulent viscosity")

fig.visu()

### Convective terms

In [ ]:
fig = plot.Graph(r"Convective terms of the standard $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_STANDARD/conv_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_STANDARD/conv_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = conv_tke_std(5e-3, xval, 5e-3)
fig.add(xval, -yval, label="Sympy TKE")
yval = conv_eps_std(5e-3, xval, 5e-3)
fig.add(xval, -yval, label="Sympy EPS")

fig.label("y","convective term")

fig.visu()


### Diffusive terms

In [ ]:
fig = plot.Graph(r"Diffusive terms of the standard $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_STANDARD/diff_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_STANDARD/diff_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = diff_tke_std(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy TKE")
yval = diff_eps_std(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"diffusive term")

fig.visu(xmin=0.0,xmax=1.0)


### Source terms

In [ ]:
fig = plot.Graph(r"Source terms of the standard $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_STANDARD/source_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_STANDARD/source_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = sources_tke_std(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy TKE")
yval = sources_eps_std(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"source term")

fig.visu(xmin=0.0,xmax=1.0)


## VDF realizable $k$-$\varepsilon$ model


### Turbulent viscosity

In [ ]:
fig = plot.Graph(r"Turbulent viscosity of the realizable $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_REALISABLE/nut.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile")

xval = numpy.linspace(0, 1, 500)
yval = nut_rea(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy")

fig.label("y","Turbulent viscosity")

fig.visu()

### Convective terms

In [ ]:
fig = plot.Graph(r"Convective terms of the realizable $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_REALISABLE/conv_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_REALISABLE/conv_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = conv_tke_rea(5e-3, xval, 5e-3)
fig.add(xval, -yval, label="Sympy TKE")
yval = conv_eps_rea(5e-3, xval, 5e-3)
fig.add(xval, -yval, label="Sympy EPS")

fig.label(r"y", r"convective term")

fig.visu(xmin=0.0, xmax=1.0)


### Diffusive terms

In [ ]:
fig = plot.Graph(r"Diffusive terms of the realizable $k$-$\varepsilon$ model in VDF at x = z = 0.005")

data = plot.loadText("./VDF_KEPS_REALISABLE/diff_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_REALISABLE/diff_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = diff_tke_rea(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy TKE")
yval = diff_eps_rea(5e-3, xval, 5e-3)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"diffusive term")

fig.visu(xmin=0.0,xmax=1.0)


### Source terms

In [ ]:
fig = plot.Graph(r"Source terms of the realizable $k$-$\varepsilon$ model in VDF at x = z = 0.005.")

data = plot.loadText("./VDF_KEPS_REALISABLE/source_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VDF_KEPS_REALISABLE/source_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = numpy.array([sources_tke_rea(5e-3, y, 5e-3) for y in xval])
fig.add(xval, yval, label="Sympy TKE")
yval = numpy.array([sources_eps_rea(5e-3, y, 5e-3) for y in xval])
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"source term")

fig.visu(xmin=0.0,xmax=1.0)


## VEF standard $k$-$\varepsilon$ model


### Turbulent viscosity

In [ ]:
fig = plot.Graph(r"Turbulent viscosity of the standard $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_STANDARD/nut.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile")

xval = numpy.linspace(0, 1, 500)
yval = nut_std(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy")

fig.label("y","Turbulent viscosity")

fig.visu()

### Convective terms

In [ ]:
fig = plot.Graph(r"Convective terms of the standard $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_STANDARD/conv_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_STANDARD/conv_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = conv_tke_std(4e-2, xval, 1/30)
fig.add(xval, -yval, label="Sympy TKE")
yval = conv_eps_std(4e-2, xval, 1/30)
fig.add(xval, -yval, label="Sympy EPS")

fig.label(r"y",r"convective term")

fig.visu(xmin=0.0,xmax=1.0)


### Diffusive terms

In [ ]:
fig = plot.Graph(r"Diffusive terms of the standard $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_STANDARD/diff_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_STANDARD/diff_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = diff_tke_std(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy TKE")
yval = diff_eps_std(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"diffusive term")

fig.visu(xmin=0.0,xmax=1.0)


### Source terms

In [ ]:
fig = plot.Graph(r"Source terms of the standard $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_STANDARD/source_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_STANDARD/source_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = sources_tke_std(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy TKE")
yval = sources_eps_std(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"source term")

fig.visu(xmin=0.0,xmax=1.0)


## VEF realizable $k$-$\varepsilon$ model


### Turbulent viscosity

In [ ]:
fig = plot.Graph(r"Turbulent viscosity of the realisable $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_REALISABLE/nut.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile")

xval = numpy.linspace(0, 1, 500)
yval = nut_rea(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy")

fig.label("y","Turbulent viscosity")

fig.visu()

### Convective terms

In [ ]:
fig = plot.Graph(r"Convective terms of the realizable $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_REALISABLE/conv_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_REALISABLE/conv_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = conv_tke_rea(4e-2, xval, 1/30)
fig.add(xval, -yval, label="Sympy TKE")
yval = conv_eps_rea(4e-2, xval, 1/30)
fig.add(xval, -yval, label="Sympy EPS")

fig.label(r"y",r"convective term")

fig.visu(xmin=0.0,xmax=1.0)


### Diffusive terms

In [ ]:
fig = plot.Graph(r"Diffusive terms of the realizable $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_REALISABLE/diff_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_REALISABLE/diff_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = diff_tke_rea(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy TKE")
yval = diff_eps_rea(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"diffusive term")

fig.visu(xmin=0.0,xmax=1.0)


### Source terms

In [ ]:
fig = plot.Graph(r"Source terms of the realizable $k$-$\varepsilon$ model in VEF at x = 0.04 and z = 0.033")

data = plot.loadText("./VEF_KEPS_REALISABLE/source_tke.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (k equation)")

data = plot.loadText("./VEF_KEPS_REALISABLE/source_eps.dat")
fig.add((data[0]),(data[1]),marker="x",label=r"TrioCFD profile (eps equation)")

xval = numpy.linspace(0, 1, 500)
yval = sources_tke_rea(4e-2, xval, 1/30)
fig.add(xval, yval, label="Sympy TKE")
yval = numpy.array([sources_eps_rea(4e-2, y, 1/30) for y in xval])
fig.add(xval, yval, label="Sympy EPS")

fig.label(r"y",r"source term")

fig.visu(xmin=0.0, xmax=1.0)


# Conclusion 
 
In this report, the coding of the diffusive, convective and source terms of the $k$ and $\varepsilon$ transport equations have been verified for the standard and realizable models. Analytical fields of velocity, $k$ and $\varepsilon$ have been imposed in input, and the terms of equations have been post-processed without resolution. The TrioCFD output profiles have been shown to be almost superimposed with the analytical profiles, except for the diffusive terms which falsely appear to be zero. This problem will soon be fixed. It should also be noted that terms involving derivatives are less accurate in VEF discretization, especially near the boundaries. A refinement of the mesh would likely result in a higher accuracy.



# References
 
[1] P.-E. Angeli et G. Damblin, Calage des paramètres du modéle $k$-$\varepsilon$ réalisable par algorithmes EGO pour la simulation des écoulements turbulents, note technique DES/ISAS/DM2S/STMF/LMSF/NT/2020-66507/A.


# Data Files

## Cas

In [ ]:
run.dumpDatasetMD("VDF_KEPS_STANDARD/Cas.data")